🧾 Project Introduction
The Air Quality Index (AQI) is a standardized indicator used to represent the quality of air in a particular region. It helps people understand how clean or polluted the air is, and what associated health effects might be a concern for them. AQI is calculated based on pollutants such as PM2.5, PM10, NO₂, SO₂, O₃, CO, etc.

Many cities in India are experiencing dangerously high pollution levels, which pose serious health risks—especially for children, the elderly, and individuals with respiratory conditions.

In [2]:
# Step 0: Install and Import Libraries

# Install the folium library (if not already installed)
!pip install folium -q

#  Import essential libraries
import pandas as pd            # For data handling
import folium                  # For interactive map creation
from google.colab import files  # For downloading files in Colab

#  Note:
# Do NOT import LayerControl from folium.plugins — it's not needed for basic usage
# from folium.plugins import LayerControl  # ← Incorrect: should be avoided unless using advanced plugins


The main objectives of this project are:

To visualize AQI data of Indian cities using an interactive map.

To categorize and color-code cities based on their AQI levels (Good, Moderate, Poor).

To raise public awareness about air quality across different regions.

To provide a clear visual aid for decision-makers and citizens to understand pollution trends.

In [4]:
# Step 1: Load the Dataset

# Prompt the user to upload the CSV file
print("/content/drive/MyDrive/indian_aqi_data.csv")
uploaded = files.upload()

# Read the uploaded file into a Pandas DataFrame
try:
    file_name = next(iter(uploaded))  # Get the uploaded file name
    print(f"\n '{file_name}' file uploaded successfully!")
    df = pd.read_csv(file_name)       # Load the file into a DataFrame
except (StopIteration, KeyError):
    print("\n No file was uploaded. Please run the cell again and upload the CSV file.")
    # Stop the code execution if upload fails
    exit()


/content/drive/MyDrive/indian_aqi_data.csv


Saving indian_aqi_data.csv to indian_aqi_data.csv

 'indian_aqi_data.csv' file uploaded successfully!


🌍 Real-World Applications
General Public can check how polluted their city is.

Government authorities can make data-driven decisions and implement better pollution control policies.

Health professionals and researchers can analyze air quality trends to study health risks and disease patterns.

Travelers and families can choose safer places based on AQI for relocation or vacation planning.



In [5]:
print("\nInitial Dataset Information:")
df.info()

print("\n------------------------------------")
print("\nMissing Values in Each Column (Before Cleaning):")
print(df.isnull().sum())

# Convert the 'AQI' column to numeric format.
# Any non-numeric values will be converted to NaN (Not a Number).
df['AQI'] = pd.to_numeric(df['AQI'], errors='coerce')

# Drop rows with any missing (NaN) values.
# This also removes rows where AQI had invalid (non-numeric) values.
df.dropna(inplace=True)

# Convert AQI values to integer type for consistency.
df['AQI'] = df['AQI'].astype(int)

print("\n------------------------------------")
print("\nDataset Information After Cleaning:")
df.info()

print("\n------------------------------------")
print("\nMissing Values in Each Column (After Cleaning):")
print(df.isnull().sum())

print("\n Dataset has been successfully cleaned.")



Initial Dataset Information:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21 entries, 0 to 20
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   City       21 non-null     object 
 1   State      21 non-null     object 
 2   Latitude   21 non-null     float64
 3   Longitude  21 non-null     float64
 4   AQI        21 non-null     int64  
dtypes: float64(2), int64(1), object(2)
memory usage: 972.0+ bytes

------------------------------------

Missing Values in Each Column (Before Cleaning):
City         0
State        0
Latitude     0
Longitude    0
AQI          0
dtype: int64

------------------------------------

Dataset Information After Cleaning:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21 entries, 0 to 20
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   City       21 non-null     object 
 1   State      21 non-null     object 
 2   Lat

In [6]:
def categorize_aqi(aqi):
    """Returns the AQI category based on the AQI value."""
    if aqi <= 100:
        return 'Good'
    elif 101 <= aqi <= 200:
        return 'Moderate'
    else:  # aqi > 200
        return 'Poor'

# Apply the function to create a new column called 'AQI_Category'
df['AQI_Category'] = df['AQI'].apply(categorize_aqi)

# Display the updated dataset with AQI categories
print("\nDataset with AQI categories added:")
print(df[['City', 'AQI', 'AQI_Category']].head())



Dataset with AQI categories added:
        City  AQI AQI_Category
0      Delhi  233         Poor
1     Mumbai  160     Moderate
2  Bengaluru   86         Good
3    Chennai  294         Poor
4    Kolkata  200     Moderate


❓ Problem Statement
Many citizens in India are unaware of the air quality levels in their own cities. As a result, they may unknowingly expose themselves to harmful pollution levels. This project aims to solve that problem by providing a simple and interactive AQI map that allows users to check AQI information city-wise.



In [7]:
# Set the approximate central coordinates of India
map_center = [20.5937, 78.9629]

# Create the Folium map object
indian_aqi_map = folium.Map(location=map_center, zoom_start=5, tiles='CartoDB positron')

# Create separate feature groups (layers) for each AQI category
good_aqi_layer = folium.FeatureGroup(name='Good (AQI ≤ 100) 🟢', show=True)
moderate_aqi_layer = folium.FeatureGroup(name='Moderate (AQI 101–200) 🟠', show=True)
poor_aqi_layer = folium.FeatureGroup(name='Poor (AQI > 200) 🔴', show=True)

# Define marker colors for each AQI category
color_map = {'Good': 'green', 'Moderate': 'orange', 'Poor': 'red'}

# Iterate through each row in the DataFrame and add a marker to the appropriate layer
for idx, row in df.iterrows():
    # Tooltip text to show on hover
    tooltip_text = f"<b>{row['City']}, {row['State']}</b><br>AQI: {row['AQI']}"

    # Create a circle marker for the city
    marker = folium.CircleMarker(
        location=[row['Latitude'], row['Longitude']],
        radius=5,
        color=color_map[row['AQI_Category']],
        fill=True,
        fill_color=color_map[row['AQI_Category']],
        fill_opacity=0.7,
        tooltip=tooltip_text
    )

    # Add the marker to the corresponding layer
    if row['AQI_Category'] == 'Good':
        marker.add_to(good_aqi_layer)
    elif row['AQI_Category'] == 'Moderate':
        marker.add_to(moderate_aqi_layer)
    else:
        marker.add_to(poor_aqi_layer)

# Add all AQI layers to the map
good_aqi_layer.add_to(indian_aqi_map)
moderate_aqi_layer.add_to(indian_aqi_map)
poor_aqi_layer.add_to(indian_aqi_map)

# Add a layer control widget to toggle filters
folium.LayerControl(collapsed=False).add_to(indian_aqi_map)

print("\n Interactive map with filter layers has been created successfully.")

# To display the map in the notebook (uncomment below if running in Jupyter or Colab)
# indian_aqi_map



 Interactive map with filter layers has been created successfully.


In [8]:
# Define the filename for the saved HTML map
map_filename = 'indian_aqi_map_with_filters.html'

# Save the map to the specified HTML file
indian_aqi_map.save(map_filename)
print(f"\n Map has been saved as '{map_filename}'.")

# Provide a download link in Google Colab
print(" Downloading the map file...")
files.download(map_filename)



 Map has been saved as 'indian_aqi_map_with_filters.html'.


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [9]:
# Sort the DataFrame by AQI in descending order and select the top 5 entries
most_polluted_cities = df.sort_values(by='AQI', ascending=False).head(5)

# Display the top 5 most polluted cities
print("\n--- Top 5 Most Polluted Cities ---")
print(most_polluted_cities[['City', 'State', 'AQI']].to_string(index=False))



--- Top 5 Most Polluted Cities ---
     City          State  AQI
Hyderabad      Telangana  300
  Chennai     Tamil Nadu  294
   Indore Madhya Pradesh  289
 Vadodara        Gujarat  282
   Kanpur  Uttar Pradesh  237


📊 Summary and Key Insights
Based on the AQI data provided:

🔴 Highly Polluted Cities (AQI > 200):
Delhi (233), Hyderabad (300), Chennai (294), Kanpur (237), Indore (289), Vadodara (282), Nashik (218)
➤ Total: 7 cities

🟠 Moderately Polluted Cities (AQI 101–200):
Mumbai, Kolkata, Pune, Lucknow, Bhopal, Agra, Visakhapatnam, Coimbatore
➤ Total: 8 cities

🟢 Good Air Quality Cities (AQI ≤ 100):
Bengaluru, Ahmedabad, Jaipur, Surat, Nagpur, Patna
➤ Total: 6 cities



✅ Recommendations
🔬 High AQI cities should implement immediate pollution control measures such as:

Odd-even traffic rules

Dust reduction campaigns

Promotion of public transport and EVs

🧪 Cities with good AQI should be encouraged to maintain and improve their clean air policies.

📢 Public awareness about AQI and health risks should be increased through mobile apps, billboards, schools, and digital platforms.

📉 Real-time and automated AQI monitoring stations should be installed in every city for accurate air quality tracking.

